In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.models import load_model
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from fractions import Fraction
import numpy as np
import datetime
from sklearn.preprocessing import MultiLabelBinarizer

Using TensorFlow backend.


In [ ]:
notes = []

midi_data = converter.parse('../vivaldi/RV_580_Concerto_for_four_violins_Op._3._No._10_-_Antonio_Vivaldi.mid')

ss = midi_data.flat

parsed_notes = ss.getElementsByClass(['Chord', 'Note'])
off_sets = [note.offset for note in parsed_notes]

note_dict = {}

for note in parsed_notes:
    offset = note.offset
    if isinstance(offset, Fraction):
            offset = numpy.round(float(offset),2)
    if note.offset in note_dict:
        if not note.isChord:
            note_dict[offset].append(note.nameWithOctave.replace('-', ''))
        else:
            note_dict[offset].extend([p.nameWithOctave.replace('-', '') for p in note.pitches])
    else:
        if not note.isChord:
            note_dict[offset] = [note.nameWithOctave.replace('-', '')]
        else:
            note_dict[offset] = [p.nameWithOctave.replace('-', '') for p in note.pitches]


for offset in note_dict:
    note_dict[offset] = list(set(note_dict[offset]))

values = note_dict.values()

values = [v for v in values]

all_uni_values = set([vv for v in values for vv in v ])

voc_len = len(all_uni_values)



mb = MultiLabelBinarizer()

all_values_binary = mb.fit_transform(values)

all_values_binary

In [ ]:
sequence_length = 32
network_input = []
network_output = []

for i in range(0, all_values_binary.shape[0] - sequence_length, 1):
    sequence_in = all_values_binary[i:i + sequence_length]
    sequence_out = all_values_binary[i + sequence_length]
    network_input.append(sequence_in)
    network_output.append(sequence_out)

network_input = np.array(network_input)
network_output = np.array(network_output)

network_input.shape

In [2]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    parsed_notes = []

    for file in np.random.choice(glob.glob("/code/dl/Classical-Piano-Composer/midi_songs_bc/*.mid"), 3):
        midi_data = converter.parse(file)

        print("Parsing %s" % file)

        ss = midi_data.flat

        notes = ss.getElementsByClass(['Chord', 'Note'])
        parsed_notes.extend(notes)
        
    off_sets = [note.offset for note in parsed_notes]

    note_dict = {}

    for note in parsed_notes:
        offset = note.offset
        if isinstance(offset, Fraction):
                offset = np.round(float(offset),2)
        if note.offset in note_dict:
            if not note.isChord:
                note_dict[offset].append(note.nameWithOctave.replace('-', ''))
            else:
                note_dict[offset].extend([p.nameWithOctave.replace('-', '') for p in note.pitches])
        else:
            if not note.isChord:
                note_dict[offset] = [note.nameWithOctave.replace('-', '')]
            else:
                note_dict[offset] = [p.nameWithOctave.replace('-', '') for p in note.pitches]


    for offset in note_dict:
        note_dict[offset] = list(set(note_dict[offset]))

    values = [v for v in note_dict.values()]
    
    mb = MultiLabelBinarizer()
    all_values_binary = mb.fit_transform(values)
    
    return all_values_binary

In [3]:
def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32
    network_input = []
    network_output = []

    for i in range(0, notes.shape[0] - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

    network_input = np.array(network_input)
    network_output = np.array(network_output)

    network_input.shape

    return (network_input, network_output)

In [6]:
notes = get_notes()

Parsing /code/dl/Classical-Piano-Composer/midi_songs_bc/1688tpvent11w.mid
Parsing /code/dl/Classical-Piano-Composer/midi_songs_bc/109.mid
Parsing /code/dl/Classical-Piano-Composer/midi_songs_bc/bwv863.mid


In [7]:
inputs, outputs = prepare_sequences(notes[:60])

In [8]:
len(notes)

1292

In [9]:
inputs.shape

(28, 32, 41)

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

In [3]:
with open('../data/training_data_bach', 'rb') as filepath:
    training_data = pickle.load(filepath)

notes = training_data['data']
mb = training_data['binarizer']

In [4]:
def create_network(network_input):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Dense(network_input.shape[2]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('../weights-improvement-18-4.0270-bigger.hdf5')

    return model
def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32
    network_input = []
    network_output = []

    for i in range(0, notes.shape[0] - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append(sequence_in)

    network_input = np.array(network_input)

    return network_input

In [5]:
network_input = prepare_sequences(notes)

In [7]:
network_input.shape

(4371, 32, 44)

In [6]:
trained_model = create_network(network_input)

In [8]:
prediction_input = network_input[np.random.choice(network_input.shape[0], 1)]
prediction_output = []

In [9]:
prediction_input.shape

(1, 32, 44)

In [9]:
preds = trained_model.predict(prediction_input, verbose=0)[0]

In [10]:
preds

array([1.90299731e-02, 1.59154751e-03, 1.12180889e-03, 3.67573411e-05,
       2.31967526e-04, 3.50651927e-02, 3.22064292e-03, 2.46833148e-03,
       1.22735102e-04, 1.29600347e-04, 2.37741560e-01, 1.27875982e-02,
       5.20595175e-04, 1.69276912e-03, 4.68761176e-02, 1.30007695e-03,
       1.54581328e-04, 1.02198726e-04, 1.59299094e-03, 3.95373791e-01,
       2.84828674e-02, 7.13372603e-04, 2.90823460e-04, 3.20669152e-02,
       4.24156040e-02, 1.45901518e-04, 3.71609858e-05, 1.90490298e-02,
       4.40639667e-02, 9.70331457e-05, 7.05256884e-04, 1.75256543e-02,
       1.33914845e-02, 6.91401001e-05, 3.09503812e-04, 3.13718757e-03,
       1.91972984e-04, 1.77080964e-03, 2.51668844e-05, 4.32203320e-04,
       1.58819207e-03, 2.75267977e-02, 4.69529862e-03, 1.07774045e-04],
      dtype=float32)

In [11]:
def get_top_p(preds):
    selected_i = []
    descend_sorted_index = np.argsort(preds)[::-1]
    preds_descend_sorted = sorted(preds, reverse=True)
    for i, pred in enumerate(preds_descend_sorted):
        print(descend_sorted_index[i], pred)
        if i==0:
            selected_i.append(descend_sorted_index[i])
        else:
            if preds_descend_sorted[i]/preds_descend_sorted[0] > 0.3:

                selected_i.append(descend_sorted_index[i])
            else:
                return selected_i

In [12]:
mb.classes_

array(['A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C#2', 'C#3',
       'C#4', 'C#5', 'C2', 'C3', 'C4', 'C5', 'C6', 'D2', 'D3', 'D4', 'D5',
       'E2', 'E3', 'E4', 'E5', 'F#2', 'F#3', 'F#4', 'F#5', 'F2', 'F3',
       'F4', 'F5', 'G#1', 'G#2', 'G#3', 'G#4', 'G#5', 'G1', 'G2', 'G3',
       'G4', 'G5'], dtype=object)

In [13]:
get_top_p(preds)

19 0.3953738
10 0.23774156
14 0.046876118


[19, 10]

In [17]:
# generate 500 notes
prediction_output = []
for note_index in range(5):
    
    pred_prob = trained_model.predict(np.array(prediction_input), verbose=0)[0]
    pred_index = get_top_p(pred_prob)
    prediction = mb.classes_[pred_index]
    
    prediction_output.append(prediction)

prediction_output = np.array(prediction_output)[1:].tolist()

19 0.3953738
10 0.23774156
14 0.046876118
19 0.3953738
10 0.23774156
14 0.046876118
19 0.3953738
10 0.23774156
14 0.046876118
19 0.3953738
10 0.23774156
14 0.046876118
19 0.3953738
10 0.23774156
14 0.046876118


In [20]:
prediction_output

[['D3', 'C#3'], ['D3', 'C#3'], ['D3', 'C#3'], ['D3', 'C#3']]

In [24]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        note = chord.Chord(pattern)
        note.offset = offset
        output_notes.append(note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    filename = 'out_{}.mid'.format(datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S'))
    midi_stream.write('midi', fp=filename)

In [28]:
create_midi(prediction_output)